In [1]:
import numpy as np
import pandas as pd
# plots
import seaborn as sns; sns.set_theme()
sns.set(palette='colorblind')
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import OrdinalEncoder

Alexandre Dias Negretti	233609	
Daniel Godoy Marques	166213	
Gyovana Mayara Moriyama	216190	

In [2]:
path = '../data/processed/'
df_train = pd.read_csv(path + 'scenario04.csv', sep=',')
df_test = pd.read_csv(path + 'scenario03.csv', sep=',')

Train -> scenario04

Test -> scenario03

# Dealing with Categorical Features - Encoding

In [ ]:
cols = ['organization', 'age', 'cnt_encounters', 'cts', 'contraceptive', 'anticoagulant', 'cnt_medications', 'cnt_procedures', 'last_encounter_dur', 'condition_dur', 
        'gender_M', 'encounterclass_ambulatory', 'encounterclass_emergency', 'encounterclass_inpatient', 'encounterclass_outpatient', 'target']

In [ ]:
df_train = pd.get_dummies(df_train, columns=['gender'], drop_first=True)
df_train = pd.get_dummies(df_train, columns=['encounterclass']) 
df_train = df_train[cols]

df_test = pd.get_dummies(df_test, columns=['gender'], drop_first=True)
df_test = pd.get_dummies(df_test, columns=['encounterclass']) 
df_test['encounterclass_emergency'] = 0
df_test = df_test[cols]
df_train.head()

,organization,age,cnt_encounters,cts,contraceptive,anticoagulant,cnt_medications,cnt_procedures,last_encounter_dur,condition_dur,gender_M,encounterclass_ambulatory,encounterclass_emergency,encounterclass_inpatient,encounterclass_outpatient,target
0,ATHOL MEMORIAL HOSPITAL,69,51,0,0,2,10,0,9,6,0,0,0,1,0,0
1,LAWRENCE GENERAL HOSPITAL,65,26,0,0,2,10,0,12,0,1,0,0,1,0,0
2,LAHEY HOSPITAL & MEDICAL CENTER BURLINGTON,32,11,0,0,2,5,0,10,0,1,0,0,1,0,0
3,UMASS MEMORIAL MEDICAL CENTER INC,53,43,0,2,2,16,0,12,2,0,0,0,1,0,0
4,EMERSON HOSPITAL -,59,25,0,0,2,9,0,9,3,1,0,0,1,0,0


In [ ]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=85)
encoder.fit(df_train[['organization']])
df_train[['organization']] = encoder.transform(df_train[['organization']])

df_test[['organization']] = encoder.transform(df_test[['organization']])

In [ ]:
df_train

,organization,age,cnt_encounters,cts,contraceptive,anticoagulant,cnt_medications,cnt_procedures,last_encounter_dur,condition_dur,gender_M,encounterclass_ambulatory,encounterclass_emergency,encounterclass_inpatient,encounterclass_outpatient,target
0,2.0,69,51,0,0,2,10,0,9,6,0,0,0,1,0,0
1,40.0,65,26,0,0,2,10,0,12,0,1,0,0,1,0,0
2,39.0,32,11,0,0,2,5,0,10,0,1,0,0,1,0,0
3,75.0,53,43,0,2,2,16,0,12,2,0,0,0,1,0,0
4,23.0,59,25,0,0,2,9,0,9,3,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,8.0,10,7,0,0,2,8,0,1,1,1,0,0,1,0,0
626,21.0,80,25,0,0,3,17,0,11,459,1,0,0,1,0,1
627,9.0,51,38,0,0,2,9,0,11,5,0,0,0,1,0,0
628,49.0,21,10,0,0,2,6,0,10,4,1,0,0,1,0,0


# Models

## Split

In [ ]:
X_train = df_train.drop(['target'], axis=1)
y_train = df_train['target']

X_test = df_test.drop(['target'], axis=1)
y_test = df_test['target']

In [ ]:
X_test

,organization,age,cnt_encounters,cts,contraceptive,anticoagulant,cnt_medications,cnt_procedures,last_encounter_dur,condition_dur,gender_M,encounterclass_ambulatory,encounterclass_emergency,encounterclass_inpatient,encounterclass_outpatient
0,85.0,52,38,0,0,0,0,0,6,389,1,0,0,1,0
1,85.0,74,40,0,0,0,0,0,0,386,1,0,0,0,1
2,85.0,33,12,0,0,0,0,0,10,1,1,0,0,1,0
3,85.0,58,27,0,0,0,0,0,12,4,1,0,0,1,0
4,85.0,58,28,0,0,0,0,0,10,4,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,85.0,33,13,0,0,0,0,0,9,3,1,0,0,1,0
66,85.0,97,79,0,0,0,0,0,5,669,1,0,0,1,0
67,85.0,44,35,0,0,0,0,0,10,0,0,0,0,1,0
68,85.0,30,22,0,0,0,0,0,13,11,1,0,0,1,0


## SMOTE

In [ ]:
oversample = SMOTE()
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

## Model Baseline

In [ ]:
predictions = np.zeros(len(y_train), dtype=int)
y_prob_n = np.zeros(len(y_train), dtype=int)

print(f'Accuracy: {accuracy_score(y_train.values, predictions)}')
print(f'AUC: {roc_auc_score(y_train.values, y_prob_n)}')
print(f'F1-Score: {f1_score(y_train.values,predictions)}')

Accuracy: 0.7841269841269841
AUC: 0.5
F1-Score: 0.0


In [ ]:
predictions = np.zeros(len(y_train_over), dtype=int)
y_prob_n = np.zeros(len(y_train_over), dtype=int)

print(f'Accuracy: {accuracy_score(y_train_over.values, predictions)}')
print(f'AUC: {roc_auc_score(y_train_over.values, y_prob_n)}')
print(f'F1-Score: {f1_score(y_train_over.values,predictions)}')

Accuracy: 0.5
AUC: 0.5
F1-Score: 0.0


## Creating models


In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import time

In [ ]:
def run_exps(X_train: pd.DataFrame , y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame) -> None:
  models = [
            ('LogReg', LogisticRegression(max_iter=500)), 
            ('RF', RandomForestClassifier(random_state=42)),
            ('LGBM', LGBMClassifier(random_state=42)),
            ('XGB', XGBClassifier(random_state=42))
          ]

  for name, model in models:
    print(f'{name}:')
    results = pd.DataFrame()

    start = time.time()
    clf = model.fit(X_train, y_train)
    end = time.time()
    results.loc[0,'Fit time'] = end - start

    start = time.time()
    y_pred = clf.predict(X_test)
    end = time.time()
    results.loc[0,'Score time'] = end - start
    y_proba = clf.predict_proba(X_test)

    results.loc[0,'Accuracy'] = accuracy_score(y_test.values, y_pred)
    results.loc[0, 'AUC'] = roc_auc_score(y_test.values, y_proba[:, 1])
    results.loc[0, 'F1-Score'] = f1_score(y_test.values, y_pred)
    display(results)
    print()

    if name == 'XGB':
      print()
      for feature, imp in zip(X_train.columns, clf.feature_importances_):
        print(f'{feature:<25} {imp}')


### Imbalanced and using all features

In [ ]:
run_exps(X_train, y_train, X_test, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.09935,0.004974,0.971429,1.0,0.933333



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.212338,0.01285,1.0,1.0,1.0



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.042415,0.003226,1.0,1.0,1.0



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.064996,0.001096,1.0,1.0,1.0




organization              0.012884595431387424
age                       0.009355081245303154
cnt_encounters            0.004791960585862398
cts                       0.0
contraceptive             0.0
anticoagulant             0.0
cnt_medications           0.004485993646085262
cnt_procedures            0.0
last_encounter_dur        0.007513192016631365
condition_dur             0.9540559649467468
gender_M                  0.0069131990894675255
encounterclass_ambulatory 0.0
encounterclass_emergency  0.0
encounterclass_inpatient  0.0
encounterclass_outpatient 0.0


### Imbalanced and using just features from original tables


In [ ]:
X_train_og = X_train[['gender_M', 'encounterclass_ambulatory', 'encounterclass_emergency', 'encounterclass_inpatient', 'encounterclass_outpatient']].copy()
X_test_og = X_test[['gender_M', 'encounterclass_ambulatory', 'encounterclass_emergency', 'encounterclass_inpatient', 'encounterclass_outpatient']].copy()

run_exps(X_train_og, y_train, X_test_og, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.013057,0.001995,0.8,0.566964,0.0



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.167917,0.014908,0.8,0.550383,0.0



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.012867,0.001983,0.8,0.589286,0.0



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.034484,0.001161,0.8,0.567602,0.0




gender_M                  0.6165420413017273
encounterclass_ambulatory 0.0
encounterclass_emergency  0.0
encounterclass_inpatient  0.3834580183029175
encounterclass_outpatient 0.0


### Imbalanced and removing condition_dur feature

In [ ]:
X_train_c = X_train.drop('condition_dur', axis=1)
X_test_c = X_test.drop('condition_dur', axis=1)

run_exps(X_train_c, y_train, X_test_c, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.160047,0.002576,0.842857,0.940051,0.685714



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.193759,0.016105,0.228571,0.878189,0.341463



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.043614,0.00247,0.728571,0.876276,0.577778



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.056209,0.001156,0.728571,0.882653,0.577778




organization              0.07369300723075867
age                       0.1602788120508194
cnt_encounters            0.04348595067858696
cts                       0.0
contraceptive             0.05491316691040993
anticoagulant             0.15029430389404297
cnt_medications           0.06572764366865158
cnt_procedures            0.0
last_encounter_dur        0.4034225642681122
gender_M                  0.04818454757332802
encounterclass_ambulatory 0.0
encounterclass_emergency  0.0
encounterclass_inpatient  0.0
encounterclass_outpatient 0.0


### With SMOTE and using all features

In [ ]:
run_exps(X_train_over, y_train_over, X_test, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.18962,0.002383,0.628571,1.0,0.518519



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.240762,0.01696,1.0,1.0,1.0



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.060655,0.002403,1.0,1.0,1.0



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.082536,0.001926,1.0,1.0,1.0




organization              0.014193600974977016
age                       0.011439659632742405
cnt_encounters            0.00665039150044322
cts                       0.0
contraceptive             0.0
anticoagulant             0.0
cnt_medications           0.01209806278347969
cnt_procedures            0.0
last_encounter_dur        0.007326679304242134
condition_dur             0.9396899342536926
gender_M                  0.008601700887084007
encounterclass_ambulatory 0.0
encounterclass_emergency  0.0
encounterclass_inpatient  0.0
encounterclass_outpatient 0.0


### With SMOTE and using just features from original tables

In [ ]:
X_train_over_og = X_train_over[['gender_M', 'encounterclass_ambulatory', 'encounterclass_emergency', 'encounterclass_inpatient', 'encounterclass_outpatient']].copy()
X_test_og = X_test[['gender_M', 'encounterclass_ambulatory', 'encounterclass_emergency', 'encounterclass_inpatient', 'encounterclass_outpatient']].copy()

run_exps(X_train_over_og, y_train_over, X_test_og, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.012913,0.001963,0.471429,0.43176,0.212766



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.188684,0.016949,0.457143,0.424107,0.208333



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.019043,0.00255,0.471429,0.434949,0.212766



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.047003,0.000962,0.471429,0.485332,0.244898




gender_M                  0.15406064689159393
encounterclass_ambulatory 0.15424005687236786
encounterclass_emergency  0.0
encounterclass_inpatient  0.11238624900579453
encounterclass_outpatient 0.5793130397796631


### With SMOTE and removing condition_dur feature

In [ ]:
X_train_over_c = X_train_over.drop('condition_dur', axis=1)
X_test_c = X_test.drop('condition_dur', axis=1)

run_exps(X_train_over_c, y_train_over, X_test_c, y_test)

LogReg:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.206824,0.002459,0.471429,0.941327,0.430769



RF:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.27375,0.017124,0.242857,0.86926,0.329114



LGBM:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.068321,0.002976,0.728571,0.889031,0.577778



XGB:


,Fit time,Score time,Accuracy,AUC,F1-Score
0,0.076757,0.001203,0.7,0.895408,0.571429




organization              0.03808746859431267
age                       0.09331975132226944
cnt_encounters            0.04372188448905945
cts                       0.0
contraceptive             0.05240983888506889
anticoagulant             0.09506525844335556
cnt_medications           0.03581303358078003
cnt_procedures            0.0
last_encounter_dur        0.4984709322452545
gender_M                  0.019696783274412155
encounterclass_ambulatory 0.123415008187294
encounterclass_emergency  0.0
encounterclass_inpatient  0.0
encounterclass_outpatient 0.0
